## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
### Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>


In [13]:
def month_lemm(month_):
    months={'января':'01','февраля':'02','января':'01','февраля':'02','марта':'03','апреля':'04','мая':'05','июня':'06',
           'июля':'07','августа':'08','сентября':'09','октября':'10','ноября':'11','декабря':'12'}

    return months.get(month_)

from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd
from pymystem3 import Mystem
from datetime import timedelta

m=Mystem()
df_habr_preview=pd.DataFrame()
KEYWORDS = ['python', 'парсинг','яндекс','так','yandex']
url_habr='https://habr.com/ru/all/'
for i in range(8):
    req=requests.get(url_habr)
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    
    for post in posts:
        preview= post.find('div', class_='post__text')
#         print(post.find('span',class_='post__time'))
#         print()
        if post.find('span',class_='post__time').text.split(' в ')[0]=='сегодня':
            date_=datetime.now().strftime("%Y-%m-%d")
        elif post.find('span',class_='post__time').text.split(' в ')[0]=='вчера':
            date_=(datetime.now()- timedelta(days=1)).strftime("%Y-%m-%d")
               
        else:
            date_=post.find('span',class_='post__time').text.split(' в ')[0]
            date_=date_.split(' ')[2]+'-'+month_lemm(date_.split(' ')[1])+'-'+date_.split(' ')[0]
        title_=post.find('a',class_='post__title_link')
        url_=post.find('a',class_='post__title_link').get('href')
        
        text_post_lemma=m.lemmatize(preview.text.lower())
        for keyword in KEYWORDS:
                if keyword in  text_post_lemma:
                    row = {'date': date_, 'title': title_, 'url': url_}
                    df_habr_preview = pd.concat([df_habr_preview, pd.DataFrame([row])])
                    break
    next_page=soup.find('a', class_='arrows-pagination__item-link_next').get('href')
    url_habr='https://habr.com/'+next_page
    
df_habr_preview.reset_index()    
df_habr_preview.head()

,date,title,url
0,2021-04-07,"[Group-IB обнаружила фишинг-киты, нацеленные н...",https://habr.com/ru/company/group-ib/blog/551236/
0,2021-04-07,[Вебинар «Ускорение на простых типах данных и ...,https://habr.com/ru/company/yandex_praktikum/b...
0,2021-04-07,[Удивительная однозвенка на MS SQL],https://habr.com/ru/post/550960/
0,2021-04-07,[KODI: собираем удобный и функциональный медиа...,https://habr.com/ru/post/551210/
0,2021-04-07,[Альтернативное собеседование на позицию разра...,https://habr.com/ru/post/551202/


### Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.



In [227]:
import json
EMAIL = ['xxx@x.ru', 'yyy@y.com']
AVAST_REQUEST='https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
data={'emailAddresses':EMAIL}
headres = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
}
res = requests.post(AVAST_REQUEST,headers=headres, json=data)
df_avast=pd.DataFrame()
x=json.loads(res.content)
breaches_json=x['breaches']
for  email, breaches in x['summary'].items():
    dd=breaches
    for _,breache in breaches.items():
        
        for id_breache in breache:
            xx= breaches_json.get(str(id_breache))
            row = {'email':email,'date': xx['publishDate'][0:10], 'site': xx['site'], 'descr':  xx['description']}
            df_avast = pd.concat([df_avast, pd.DataFrame([row])])
df_avast.reset_index()
print(df_avast) 


       email        date                  site  \
0   xxx@x.ru  2017-02-14        parapa.mail.ru   
0   xxx@x.ru  2016-10-29                vk.com   
0   xxx@x.ru  2016-10-21             adobe.com   
0   xxx@x.ru  2017-02-14         cfire.mail.ru   
0   xxx@x.ru  2017-01-31      cdprojektred.com   
0   xxx@x.ru  2016-10-23             imesh.com   
0  yyy@y.com  2020-01-03         azcentral.com   
0  yyy@y.com  2020-05-28           wishbone.io   
0  yyy@y.com  2017-11-04        myheritage.com   
0  yyy@y.com  2021-02-11  forums.vkmonline.com   
0  yyy@y.com  2019-06-13             canva.com   
0  yyy@y.com  2016-10-24           dropbox.com   
0  yyy@y.com  2016-10-21          linkedin.com   
0  yyy@y.com  2017-03-01          rayli.com.cn   
0  yyy@y.com  2019-10-17             zynga.com   
0  yyy@y.com  2016-10-21             adobe.com   
0  yyy@y.com  2018-02-18            netlog.com   
0  yyy@y.com  2017-03-15        globalreach.eu   
0  yyy@y.com  2016-10-23             imesh.com   
